In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json

In [2]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
CIRCOM_PRIME = 21888242871839275222246405745257275088548364400416034343698204186575808495617
MAX_POSITIVE = CIRCOM_PRIME // 2
MAX_NEGATIVE = MAX_POSITIVE + 1 # The most positive number

EXPONENT = 15

def from_circom(x):
    if type(x) != int:
        x = int(x)
    if x > MAX_POSITIVE: 
        return x - CIRCOM_PRIME
    return x
    
def to_circom(x):
    if type(x) != int:
        x = int(x)
    if x < 0:
        return x + CIRCOM_PRIME 
    return x

class SeparableConv2D(nn.Module):
    '''Separable convolution'''
    def __init__(self, in_channels, out_channels, stride=1):
        super(SeparableConv2D, self).__init__()
        self.dw_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pw_conv =  nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, x):
        x = self.dw_conv(x)
        x = self.pw_conv(x)
        return x

In [3]:
input = torch.randn((1, 3, 5, 5))
model = SeparableConv2D(3, 6)

In [4]:
def DepthwiseConv(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    assert(nFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    # out = np.zeros((outRows, outCols, nFilters))
    out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    # remainder = np.zeros((outRows, outCols, nFilters))
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        out[row][col][channel] += int(input[row*strides+x, col*strides+y, channel]) * int(weights[x, y, channel])
                
                out[row][col][channel] += int(bias[channel])
                remainder[row][col][channel] = str(int(out[row][col][channel] % n))
                out[row][col][channel] = int(out[row][col][channel] // n)
                            
    return out, remainder

In [5]:
weights = model.dw_conv.weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()

expected = model.dw_conv(input).detach().numpy()

padded = F.pad(input, (1,1,1,1), "constant", 0)
padded = padded.squeeze().numpy().transpose((1, 2, 0))
weights = weights.transpose((1, 2, 0))

quantized_image = padded * 10**EXPONENT
quantized_weights = weights * 10**EXPONENT

actual, rem = DepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_weights.round(), bias)

expected = expected.squeeze().transpose((1, 2, 0))
expected = expected * 10**EXPONENT

assert(np.allclose(expected, actual, atol=0.00001))

In [6]:
weights = model.dw_conv.weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()

padded = F.pad(input, (1,1,1,1), "constant", 0)
padded = padded.squeeze().numpy().transpose((1, 2, 0))
weights = weights.transpose((1, 2, 0))

quantized_image = padded * 10**EXPONENT
quantized_weights = weights * 10**EXPONENT

out, remainder = DepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_weights.round(), bias)

circuit_in = quantized_image.round().astype(int).astype(str).tolist()
circuit_weights = quantized_weights.round().astype(int).astype(str).tolist()
circuit_bias = bias.round().astype(int).astype(str).tolist()

input_json_path = "depthwiseConv2D_input.json"
with open(input_json_path, "w") as input_file:
    json.dump({"in": circuit_in,
               "weights": circuit_weights,
               "remainder": remainder,
               "out": out,
               "bias": circuit_bias,
              },
              input_file)